In [1]:
require "nn"
require "optim"

## Step 1: Building Architecture

**Building Model 1: a two layer convolutional neural network**

In [2]:
function initialize_model_1()
    --container
    model = nn.Sequential()
    --first layer
    ----Convolution
    model:add(nn.SpatialConvolutionMM(1, 32, 5, 5))
    ----Nonlinearity
    model:add(nn.ReLU())
    ----Max-pooling
    model:add(nn.SpatialMaxPooling(2, 2))
    --Second layer
    ----Convolution
    model:add(nn.SpatialConvolutionMM(32, 64, 5, 5))
    ----Nonlinearity
    model:add(nn.ReLU())
    ----Max-Pooling
    model:add(nn.SpatialMaxPooling(2, 2))
    ----Reshape Results from the second layer
    model:add(nn.Reshape(64 * 5 *5))
    --Output layer: linear classification
    model:add(nn.Linear(64 * 5 * 5, 10))
    
    return model
end

**Building Model 2: a one layer convolutional neural network**

In [3]:
function initialize_model_2()
    --container
    model = nn.Sequential()
    --first layer
    ----Convolution
    model:add(nn.SpatialConvolutionMM(1, 32, 5, 5))
    ----Nonlinearity
    model:add(nn.ReLU())
    ----Max-pooling
    model:add(nn.SpatialMaxPooling(2, 2))
    ----Reshape Results from the second layer
    model:add(nn.Reshape(32 * 14 * 14))
    --Output layer: linear classification
    model:add(nn.Linear(32 * 14 * 14, 10))
    
    return model
end

**Building Model 3:only linear classification layer**

In [4]:
function initialize_model_3()
    model = nn.Sequential()
    model:add(nn.Reshape(1 * 32 * 32))
    model:add(nn.Linear(1 * 32 * 32, 10))
    return model
end

## Step 2: Train the Model

In [5]:
-- load data
require "batches"
require 'gnuplot'
mnistTrain = torch.load("./data/trainingData.t7") 
mnistTest  = torch.load("./data/testData.t7")

In [6]:
training_step = 1000
train_batch_size = 100
test_batch_size  = 1000
inspection_step = 50

In [7]:
testImages, testLabels = mnistTest:getNextBatch(test_batch_size) 
function getModelAccuracy (data, model) 
    local preds = model:forward(testImages) 
    return accuracy(preds, testLabels)
end

In [8]:
function trainModel (data, model, batch_size, learning_rate)
    --load image
    images, labels = data:getNextBatch(batch_size)
    --feedforward
    scores = model:forward(images)
    --define cross entropy criterion
    crit = nn.CrossEntropyCriterion()
    --use crit to calculate teh loss function
    loss = crit:forward(scores, labels)
 
    --backward
    --calculate gradient of loss w.r.t scores
    dScores = crit:backward(scores, labels) 
    --find the rest of the gradients
    model:backward(images, dScores) 
    --update parameters 
    model:updateParameters(learning_rate)
    --zero the gradients
    model:zeroGradParameters()
    --return model
    return model
end

In [9]:
function trainModel_L1 (data, model, batch_size, learning_rate)
    --load image
    images, labels = data:getNextBatch(batch_size)
    --retrieve the model's parameters and gradients
    parameters,gradParameters = model:getParameters()
    --print('para: '..torch.norm(parameters,1))
 
    --zero the gradients
    model:zeroGradParameters()
    
    --feedforward
    scores = model:forward(images)
    --define cross entropy criterion
    crit = nn.CrossEntropyCriterion()
    --use crit to calculate teh loss function
    loss = crit:forward(scores, labels)
    
    --backward
    --calculate gradient of loss w.r.t scores
    dScores = crit:backward(scores, labels) 
    --find the rest of the gradients
    model:backward(images, dScores) 
    --update parameters 

    local feval = function(x)
        -- get new parameters
        if x ~= parameters then
            parameters:copy(x)
        end
        -- define L1 coef 
        coefL1 = 1e-3;
        local norm,sign= torch.norm,torch.sign
        --print('old loss'..loss)
        loss = loss + norm(coefL1 * parameters,1)
        --print('adjusted loss: '..loss)
        gradParameters:add( sign(parameters):mul(coefL1) )
        --print('L1 grad: '..norm(gradParameters,1))
        return loss, gradParameters
    end;
    
    sgdState = {
            learningRate = 0.05,
            momentum = 0,
            learningRateDecay = 0
         }
    optim.sgd(feval, parameters, sgdState)
   
    --return model
    return model
end

**Train Model 1: a two layer CNN**

In [12]:
--Test Model 1
--::redo::
model = initialize_model_1()
accuracy_inspection = torch.zeros(training_step / inspection_step)
for i = 1,training_step do 
    model = trainModel_L1(mnistTrain, model, train_batch_size, learning_rate)
    if (i % inspection_step == 0) then
        accuracy_inspection[i / inspection_step] = getModelAccuracy(mnistTest, model)
        print('['..tonumber(i / training_step * 100)..'%]')
        --print('['..tonumber(i / training_step * 100)..'%]')
        print(accuracy_inspection[i / inspection_step])
        parameters,gradParameters = model:getParameters()
        print('L1 grad: '..torch.norm(gradParameters,1))
        if (accuracy_inspection[i / inspection_step] < 0.13) then
            print('Bad Luck, Do it Again')
            --goto redo
            --break
        end
        
    end
end
range = torch.range(inspection_step, training_step, inspection_step)
trained_model_1 = model
trained_model_1_acc = accuracy_inspection
torch.save('model1.t7', trained_model_1)
torch.save('model1_acc.t7', trained_model_1_acc)

[5%]	
0.785	


L1 grad: 472.23898055948	


[10%]	
0.857	


L1 grad: 606.17457178172	


[15%]	
0.858	


...

L1 grad: 235.6337464715	


[95%]	
0.958	


L1 grad: 415.56455486533	


[100%]	
0.947	


L1 grad: 461.96493764846	


**Train Model 1: a one layer CNN**

In [11]:
--Test Model 2
::redo::
model = initialize_model_2()
accuracy_inspection = torch.zeros(training_step / inspection_step)
for i = 1,training_step do 
    model = trainModel_L1(mnistTrain, model, train_batch_size, learning_rate)
    if (i % inspection_step == 0) then
        accuracy_inspection[i / inspection_step] = getModelAccuracy(mnistTest, model)
        print('['..tonumber(i / training_step * 100)..'%]')
        print(accuracy_inspection[i / inspection_step])
        if (accuracy_inspection[i / inspection_step] < 0.12) then
            print('Bad Luck, Do it Again')
            --goto redo
            break
        end
        parameters,gradParameters = model:getParameters()
        print('L1 grad: '..torch.norm(gradParameters,1))
    end
end
range = torch.range(inspection_step, training_step, inspection_step)
trained_model_2 = model
trained_model_2_acc = accuracy_inspection
torch.save('model2.t7', trained_model_2)
torch.save('model2_acc.t7', trained_model_2_acc)

[5%]	
0.848	


L1 grad: 470.01260191718	


[10%]	
0.879	


L1 grad: 405.80498595691	


[15%]	
0.882	


...

L1 grad: 294.25950755272	


[95%]	
0.934	


L1 grad: 363.95813796223	


[100%]	
0.929	


L1 grad: 492.83908237513	


**Train Model 3: a linear classifier**

In [10]:
--Test Model 3
model = initialize_model_3()
accuracy_inspection = torch.zeros(training_step / inspection_step)
for i = 1,training_step do 
    model = trainModel_L1(mnistTrain, model, train_batch_size, learning_rate)
    if (i % inspection_step == 0) then
        accuracy_inspection[i / inspection_step] = getModelAccuracy(mnistTest, model)
        print('['..tonumber(i / training_step * 100)..'%]')
        print(accuracy_inspection[i / inspection_step])
    end
end
range = torch.range(inspection_step, training_step, inspection_step)
trained_model_3 = model
trained_model_3_acc = accuracy_inspection
torch.save('model3.t7', trained_model_3)
torch.save('model3_acc.t7', trained_model_3_acc)

[5%]	
0.816	


[10%]	
0.846	


[15%]	
0.861	


[20%]	
0.867	


[25%]	
0.871	


...


0.887	


[85%]	
0.891	


[90%]	
0.896	


[95%]	
0.894	


[100%]	
0.895	
